<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Parking_spot_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Car Parking Slot Dataset](https://drive.google.com/file/d/11DyZ165lZGzULEZSQNofyy9A8xaYgFJ2)

In [8]:
import os
import shutil
import sklearn
import skimage
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn import ensemble
from sklearn import model_selection

In [ ]:
zip_folder = '/content/drive/MyDrive/DATA/car-parking-slot-empty-notempty.zip'
shutil.copyfile(zip_folder, '/content/data.zip')
!unzip \*.zip && rm *.zip

In [3]:
data = []
labels = []
for id, cat in enumerate(['empty', 'not_empty']):
  path = f'./clf-data/{cat}/'
  for img in os.listdir(path):
    img = skimage.io.imread(path+img)
    img = skimage.transform.resize(img, (15, 15))
    data.append(img.flatten())
    labels.append(id)

data = np.asarray(data)
labels = np.asarray(labels)

In [4]:
data.shape,labels.shape

((6090, 675), (6090,))

In [7]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(data, labels, test_size=0.2, random_state=32)

In [ ]:
train_x.shape, test_x.shape,

In [ ]:
clf = sklearn.svm.SVC()

parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}]

grid_search = sklearn.ensemble.GridSearchCV(clf, parameters)

In [ ]:
grid_search.fit(train)


grid_search = GridSearchCV(classifier, parameters)

grid_search.fit(x_train, y_train)

# test performance
best_estimator = grid_search.best_estimator_

y_prediction = best_estimator.predict(x_test)

score = accuracy_score(y_prediction, y_test)

print('{}% of samples were correctly classified'.format(str(score * 100)))

pickle.dump(best_estimator, open('./model.p', 'wb'))